# Retrieval-Based Voice Conversion (RVC)

## Install

In [ ]:
# Installation - only needs to run once
%cd ~
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
!mv Retrieval-based-Voice-Conversion-WebUI RVC
%cd RVC
print("Installing...")
!pip install -q -r requirements.txt

!python tools/download_models.py

!pip install -q pyngrok
!mkdir ~/RVC/myaudio
!mkdir ~/RVC/ffmpeg
!wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -O ~/RVC/ffmpeg/ffmpeg-release.tar.xz
!tar -xvf ~/RVC/ffmpeg/ffmpeg-release.tar.xz -C ~/RVC/ffmpeg/
!mv ~/RVC/ffmpeg/ff*/* ~/RVC/ffmpeg


#### Set up ffmpeg

In [ ]:
# Link ffmpeg - run once each time the instance restarts
!unlink /opt/conda/bin/ffmpeg 
!unlink /opt/conda/bin/ffprobe 
!ln -s /home/studio-lab-user/RVC/ffmpeg/ffmpeg /opt/conda/bin/ffmpeg 
!ln -s /home/studio-lab-user/RVC/ffmpeg/ffprobe /opt/conda/bin/ffprobe 

---

# Start the WebUI

A quick guide on using Pinggy and Zrok - https://youtu.be/QA2o1bHt9U4

### Start with Pinggy

If the connection errors out, reload the page. Free tier users will have to restart the WebUI code cell once per hour. If you're training a model, wait for the training to complete before restarting the cell.

In [ ]:
# Install SSH (only needs to run once)
!conda config --add channels conda-forge
!conda config --set channel_priority strict
!conda install -y openssh

In [ ]:
# Start the WebUI with Pinggy
%cd /home/studio-lab-user/RVC

from multiprocessing import Process
import sys
import time

open('log.txt', 'w').close()

def run_app():
    !python infer-web.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7865 a.pinggy.io > log.txt
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

### Start with Zrok

In [ ]:
# Install Zrok (only needs to run once)

!mkdir /home/studio-lab-user/zrok
%cd /home/studio-lab-user/zrok
!wget https://github.com/openziti/zrok/releases/download/v0.4.23/zrok_0.4.23_linux_amd64.tar.gz
!tar -xvf ./zrok*.gz 
!chmod a+x /home/studio-lab-user/zrok/zrok 


In [ ]:
# --- Create a Zrok account ---

# Enter your email address in the email variable

email = '####@gmail.com' # replace with your email

# -----------------------------

cmd = '/home/studio-lab-user/zrok/zrok invite'
log = '/home/studio-lab-user/zrok/log.txt'

!pip install pexpect
!touch $log

import pexpect
import time
child = pexpect.spawn('bash')
child.sendline(f'{cmd} | tee {log}')
child.expect('enter and confirm your email address...')
time.sleep(1); child.sendline(email); time.sleep(1); child.send(chr(9)); time.sleep(1)
child.sendline(email); time.sleep(1); child.send('\n'); time.sleep(1); child.send(chr(9))
time.sleep(1); child.send('\r\n'); time.sleep(2); child.close()
!cat $log
!rm $log

In [ ]:
# Enable Zrok (only neeeds to run once)
# Paste your Zrok token in the token variable
# You can find the token by visiting you Zrok account page, under the Detail tab
# https://api.zrok.io

token = ''

!/home/studio-lab-user/zrok/zrok enable $token

In [ ]:
# Start the WebUI with Zrok

%cd /home/studio-lab-user/RVC
!python infer-web.py & /home/studio-lab-user/zrok/zrok share public http://localhost:7865 --headless

In [ ]:
# Disable Zrok
# This only needs to run if you want to use a new Zrok token

!/home/studio-lab-user/zrok/zrok disable

### Start with ngrok

In [ ]:
# --- Variables ---

ngrok_token = "" 
# Put your ngrok token here (obtainable from https://ngrok.com)
# Example: Ngrok_token = "2Fw13n4GcJT12g7mSDUC62cdNGb_5svdjf3Gg5vfhr4nGr5gF"

ngrok_domain = "" # optional, leave empty if you don't have a domain
# -----------------

%cd /home/studio-lab-user/RVC

!pip install pyngrok

port = 7865
code = "python infer-web.py"

from pyngrok import ngrok, conf
import fileinput
import sys
import gc

gc.collect()

if ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(port , pyngrok_config=conf.PyngrokConfig(auth_token=ngrok_token),
                    bind_tls=True, domain=ngrok_domain).public_url
  print("\n😁* Your URL is: *😁")
  print(srv)
  print ("😁****************😁\n")
  get_ipython().system(f"python  -c '{code}'")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

---

# Download a Model

### Download .pth and .index files

In [ ]:
pth_url = "https://huggingface.co/sail-rvc/SpongeBob_SquarePants__RVC_v2_/resolve/main/model.pth?download=true"
index_url = "https://huggingface.co/sail-rvc/SpongeBob_SquarePants__RVC_v2_/resolve/main/model.index?download=true"
name = "spongebob"

%cd /home/studio-lab-user/RVC
import os
os.makedirs('assets/weights',exist_ok=True)
os.makedirs('assets/indices', exist_ok=True)
get_ipython().system(f'wget "{pth_url}" -O "assets/weights/{name}.pth"')
get_ipython().system(f'wget "{index_url}" -O "assets/indices/{name}.index"')

### Download .zip file
This will only work if the .pth and .index files are in the root folder in the zip file
If these files are in another folder, please unzip the file locally, and upload the .pth and .index files to the checkpoints folder using the File Browser

In [ ]:
zip_url = "https://huggingface.co/sxndypz/rvc-v2-models/resolve/main/kiritan.zip?download=true"
name = "kiritan"
# ---------------------------- #

%cd /home/studio-lab-user/RVC
import os
os.makedirs('assets/weights',exist_ok=True)
os.makedirs('assets/indices', exist_ok=True)
get_ipython().system(f'wget "{zip_url}" -O "/tmp/{name}.zip"')
!mkdir /tmp/$name
%cd /tmp
get_ipython().system(f'unzip "/tmp/{name}.zip" -d "/tmp/{name}/"')
!mv /tmp/{name}/*.pth /home/studio-lab-user/RVC/assets/weights/
!mv /tmp/{name}/*.index /home/studio-lab-user/RVC/assets/indices/

---

# Merge vocals with the instrumental track

In [ ]:
audio_vocals = "/tmp/gradio/a39b8487c6167376b9a6c5cd17a0f86b8d2c43de/audio.wav"
audio_instrumental = "opt/instrument_mysong.mp3_10.wav"
out = "fullsong.wav"

%cd /home/studio-lab-user/RVC

!mkdir ~/RVC/audio-out

!ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i "$audio_vocals" -i "$audio_instrumental" -c:v copy -filter_complex "[0:a][1:a] amix=inputs=2:duration=longest" ~/RVC/audio-out/$out